<a href="https://colab.research.google.com/github/Sonma505/test-run/blob/main/BDSiC_Day_5_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GWAS QC Pipeline
- General DS issues:
  1. Removing inaccurate data -- context dependent
  2. Identifying outliers -- exploratory visualization
  3. Addressing missing data -- we will work through various examples of this
  4. Removing duplicated data -- we work through an example of this
  5. Standardizing data formats: remove unnecessary columns, convert data types
  6. Ensuring that your data fits the assumptions of your model

---
- Review: Specific GWAS issues:
  1. Shared ancestry
  2. LD: association not causation

---

(From BDSiC_5A)
Each particular dataset and question will develop a pipeline to address unique challenges that arise from unique aspects of the data/question. Genomic databases are a great example of a specialized pipeline to reduce the bias of non-independence on top of the more typical missing or inaccurate data grooming:

       a. removing rare or monomorphic variants - minor allele frequency
       b. filtering missing SNPs
       c. identifying and removing genotyping errors
           * heterozygosity
           * sex discrepancy
           * removing variants that are not in Hardy-Weinberg equilibrium (which can indicate: genotyping errors, batch effects, population stratification)
           * PCA is straightforward way to identify population stratification or batch effects
       d. accounting for ancestry and relatedness - another way population stratification pops up! Cases and controls should be matched by ancestry to avoid confounding and, therefore, false positives.
       e. account for false positives (you are testing millions of hypotheses simultaneously)  

    * The program Plink is standard for analysis GWAS and it requires files of specific formats: bed, fam, bim files all contain slightly different information, including gneomic variants, and family relationships. Plink makes the quality control step easier (a huge challenge) with built-in functions for the above. There are python packages that wrap around Plink and multivariate analysis tools to create a seamless pipeline.

# representing the 3-D world in 2-D: Lessons of the Mercator projection
Benefits of Mercator projection on maps:
* Rhumb lines are straight so navigation is easier
Cons:
* Northern land masses look HUGE and Southern land masses look tiny

More accurate:
* https://www.discovery.com/science/AuthaGraph-World-Accurate-Map
* Gall-Peters projection

Think about Subway maps? Are they accurate representations of the cityscape or are they not (and if not, why do they exist?)

* __There are ALWAYS DELIBERATE decisions that we make about what to emphasize__

# Data will speak for itself, when it cleans itself.
- What is "tidy data"? https://vita.had.co.nz/papers/tidy-data.pdf
- Adopting these principles when creating your own datasets will save you so much time. However, we are usually using someone else's data and they probably didn't adopt these principles.
- Nicely bulleted principles here: https://kbroman.org/dataorg/

# General Principles of Data Cleaning, Data Preparation & QC:
* The secret shame of Data Science: Data preparation is tedious, requires context, requires coding skills, demands creativity, doesn't feel like progress :(
* DOCUMENT, DOCUMENT, DOCUMENT your process
  - NOT just WHAT, but WHY. You're not going to remember the justifications tomorrow, let alone in a week or month or year. Help out your future self!
  - no one is 'good' at this, but it is crucial.
  - Trust no-one, not even (past) yourself -- (future) self
---
## Fundamentals
1. Removing inaccurate data
2. Identifying outliers
3. Addressing missing data
4. Removing duplicated data
5. Standardizing data formats (especially dates!): remove unnecessary columns, convert data types, check merged columns/rows
6. Ensuring that your data fits the assumptions of your model
---
## Questions:
1. Is this data what I expect, even when I consider things that could have gone wrong in the data collection process? What data patterns would be present, if various common things had gone wrong?
2. Have I checked my calculations?
3. Can I explain outliers/oddities? Context matters: these could be real, important values, or they could be artifacts, or they could be data entry mistakes.
4. Does my process/pipeline make sense on a broad scale?


In [ ]:
# typical Data Cleaning stack
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OrdinalEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
# Example 1: we will use this to see two approaches to missing data.
# mostly follows this: https://github.com/sumony2j/Data_Cleaning_Preprocessing/blob/main/Sample_Data_Cleaning%26Preprocessing.ipynb

df1 = pd.read_csv('./Simple_Data_Ex1.csv')
df1
# This is a small data set so we can visually see if there are any outliers or 'strange' data.
# however, since we can't talk to the creators of this data set it is challenging to determine accuracy, outside
# of is the data generally 'reasonable'?

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [ ]:
# how much missing data is there in each of these columns?
df1.isnull().sum()

,0
Country,0
Age,1
Salary,1
Purchased,0


In [ ]:
# how do we want to deal with missing data? There are a handful of common strategies:
# 0, distinctive value (-999), or mean of the column
# ---------
# 1. A solid choice is to replace the NaN with 0.0
# we can do that with the SimpleImputer that we imported from the sklearn.impute module:
Imputer1 = SimpleImputer(missing_values=np.nan,strategy='constant',fill_value=0)
# apply the results to the df1 but only on columns that have missing data as counted in the cell above:
#columns 1 and 2.
df1.iloc[:,1:3]=Imputer1.fit_transform(df1.iloc[:,1:3])

In [ ]:
# let's check in again on our dataframe
df1

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,0.0,Yes
5,France,35.0,58000.0,Yes
6,Spain,0.0,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [ ]:
#2. mean of the column
# note: since the data is no longer missing as we just modified it above to be 0, we need to
# re-read in the data set to get this to work.
df1b = pd.read_csv('./Simple_Data_Ex1.csv')
df1b
Imputer2=SimpleImputer(missing_values=np.nan,strategy='mean')
df1b.iloc[:,1:3]=Imputer2.fit_transform(df1b.iloc[:,1:3])

In [ ]:
# let's check in again:
df1b
round(df1b,2)

,Country,Age,Salary,Purchased
0,France,44.00,72000.00,No
1,Spain,27.00,48000.00,Yes
2,Germany,30.00,54000.00,No
3,Spain,38.00,61000.00,No
4,Germany,40.00,63777.78,Yes
5,France,35.00,58000.00,Yes
6,Spain,38.78,52000.00,No
7,France,48.00,79000.00,Yes
8,Germany,50.00,83000.00,No
9,France,37.00,67000.00,Yes


In [ ]:
# Here is a second dataset. It is a bit larger than the last example.
# taken from here: https://github.com/sumony2j/Data_Cleaning_Preprocessing/blob/main/Airbnb_NY_Data_Cleaning%26Preprocessing.ipynb
df2 = pd.read_csv('./Data_example2.csv')
# we just want to peak at the data - since it is large
df2.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [ ]:
# how big is the data?
df2.shape
# Notice the difference between an attribute -- like shape -- and a method?
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [ ]:
# let's see how many missing data pieces there are in each column.
df2.isnull().sum()

,0
id,0
name,16
host_id,0
host_name,21
neighbourhood_group,0
neighbourhood,0
latitude,0
longitude,0
room_type,0
price,0


In [ ]:
# some of these columns are not useful to answer our particular question(s).
# we can get rid of columns like so:
df2.drop(['name','host_name','last_review'],axis=1,inplace=True)

In [ ]:
#let's peek again:
print(df2.head())
# see where any remaining missing data is in the columns that we decided to keep
df2.isnull().sum()

     id  host_id neighbourhood_group neighbourhood  latitude  longitude  \
0  2539     2787            Brooklyn    Kensington  40.64749  -73.97237   
1  2595     2845           Manhattan       Midtown  40.75362  -73.98377   
2  3647     4632           Manhattan        Harlem  40.80902  -73.94190   
3  3831     4869            Brooklyn  Clinton Hill  40.68514  -73.95976   
4  5022     7192           Manhattan   East Harlem  40.79851  -73.94399   

         room_type  price  minimum_nights  number_of_reviews  \
0     Private room    149               1                  9   
1  Entire home/apt    225               1                 45   
2     Private room    150               3                  0   
3  Entire home/apt     89               1                270   
4  Entire home/apt     80              10                  9   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               0.21                               6               365  
1               0.

,0
id,0
host_id,0
neighbourhood_group,0
neighbourhood,0
latitude,0
longitude,0
room_type,0
price,0
minimum_nights,0
number_of_reviews,0


In [ ]:
# instead of mean, median or 0.0 value, we could replace mssing values with the most frequent value
imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
# or we could use the strategy='constant',fill_value=0
df2[['reviews_per_month']]=imputer.fit_transform(df2[['reviews_per_month']])
df2.head(25)

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
1,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
2,3647,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,0.02,1,365
3,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
4,5022,7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0
5,5099,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,0.59,1,129
6,5121,7356,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45,49,0.40,1,0
7,5178,8967,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,3.47,1,220
8,5203,7490,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2,118,0.99,1,0
9,5238,7549,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,1.33,4,188


In [ ]:
neighbourhood=pd.DataFrame(df2['neighbourhood'])
neighbourhood

,neighbourhood
0,Kensington
1,Midtown
2,Harlem
3,Clinton Hill
4,East Harlem
...,...
48890,Bedford-Stuyvesant
48891,Bushwick
48892,Harlem
48893,Hell's Kitchen


In [ ]:
df2_unique_neighbourhoods=df2["neighbourhood"].unique()
df2_unique_neighbourhoods

array(['Kensington', 'Midtown', 'Harlem', 'Clinton Hill', 'East Harlem',
       'Murray Hill', 'Bedford-Stuyvesant', "Hell's Kitchen",
       'Upper West Side', 'Chinatown', 'South Slope', 'West Village',
       'Williamsburg', 'Fort Greene', 'Chelsea', 'Crown Heights',
       'Park Slope', 'Windsor Terrace', 'Inwood', 'East Village',
       'Greenpoint', 'Bushwick', 'Flatbush', 'Lower East Side',
       'Prospect-Lefferts Gardens', 'Long Island City', 'Kips Bay',
       'SoHo', 'Upper East Side', 'Prospect Heights',
       'Washington Heights', 'Woodside', 'Brooklyn Heights',
       'Carroll Gardens', 'Gowanus', 'Flatlands', 'Cobble Hill',
       'Flushing', 'Boerum Hill', 'Sunnyside', 'DUMBO', 'St. George',
       'Highbridge', 'Financial District', 'Ridgewood',
       'Morningside Heights', 'Jamaica', 'Middle Village', 'NoHo',
       'Ditmars Steinway', 'Flatiron District', 'Roosevelt Island',
       'Greenwich Village', 'Little Italy', 'East Flatbush',
       'Tompkinsville', 'Asto

In [ ]:
#https://github.com/durgenious/loan_data_processing
# use data loan.csv
df3 = pd.read_csv('loan.csv')  # load 'loan.csv'
df3.head(5) # displays first 5 records in the DataFrame
# df3.tail(5) # displays last 5 records in the DataFrame

,UID,Marital_status,Dependents,Is_graduate,Income,Loan_amount,Term_months,Credit_score,approval_status,Age,Sex,Purpose,Hobby
0,LP001002,NO,0.0,Graduate,45848,NaN,360.0,1.0,0,40.0,Male,Education,Reading
1,LP001002,NO,0.0,Graduate,45848,NaN,360.0,1.0,0,40.0,Male,Education,Reading
2,LP001003,YES,1.0,Graduate,15325,128.0,360.0,1.0,1,22.0,Male,Education,Reading
3,LP001005,YES,0.0,Graduate,29105,66.0,360.0,1.0,0,27.0,Male,Education,Reading
4,LP001006,YES,0.0,Not Graduate,42944,120.0,360.0,1.0,0,36.0,F,Education,Reading


In [ ]:
print(df3.info())
print("------")
print(df3.describe())
print("~~~~~~")
print(df3.shape)
print("**********")
print(df3.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   UID              615 non-null    object 
 1   Marital_status   613 non-null    object 
 2   Dependents       600 non-null    float64
 3   Is_graduate      613 non-null    object 
 4   Income           615 non-null    int64  
 5   Loan_amount      592 non-null    float64
 6   Term_months      601 non-null    float64
 7   Credit_score     565 non-null    float64
 8   approval_status  615 non-null    int64  
 9   Age              609 non-null    float64
 10  Sex              602 non-null    object 
 11  Purpose          615 non-null    object 
 12  Hobby            615 non-null    object 
dtypes: float64(5), int64(2), object(6)
memory usage: 62.6+ KB
None
------
       Dependents        Income  Loan_amount  Term_months  Credit_score  \
count  600.000000  6.150000e+02   592.000000   601.0000

In [ ]:
# drop duplicate rows
dup = df3.duplicated(keep='first')   # Find duplicate rows in the DataFrame 'df' based on all columns, keeping the first occurrence.
dropped = df3.loc[dup]   # DataFrame 'dropped' containing only the duplicated rows.
dropped['UID'].unique()[0]    # Extract the unique identifier ('UID') of the duplicated rows.

'LP001002'

In [ ]:
df3.loc[df3['UID'].isin(dropped['UID'].unique())]   # Filter the original DataFrame to show only rows with the same 'UID' as the duplicated rows.

,UID,Marital_status,Dependents,Is_graduate,Income,Loan_amount,Term_months,Credit_score,approval_status,Age,Sex,Purpose,Hobby
0,LP001002,NO,0.0,Graduate,45848,NaN,360.0,1.0,0,40.0,Male,Education,Reading
1,LP001002,NO,0.0,Graduate,45848,NaN,360.0,1.0,0,40.0,Male,Education,Reading


In [ ]:
df3.drop_duplicates(inplace=True)  # drop duplicate rows from the original DataFrame
df3.shape  # shape of the DataFrame after dropping duplicates (1 duplicate)

(614, 13)

In [ ]:
dup2 = df3.duplicated(subset=['UID'], keep='first')    # Find duplicate rows in the DataFrame 'df' based on 'UID', keeping the first occurrence.
dropped2 = df3.loc[dup2]
dropped2['UID'].unique()[0]   # Extract the unique identifier ('UID') of the duplicated rows.

'LP002872'

In [ ]:
df3.loc[df3['UID'].isin(dropped2['UID'].unique())]    # Filter the original DataFrame to show only rows with the same 'UID' as the duplicated rows.

,UID,Marital_status,Dependents,Is_graduate,Income,Loan_amount,Term_months,Credit_score,approval_status,Age,Sex,Purpose,Hobby
577,LP002872,Yes,0.0,Graduate,45672,136.0,360.0,0.0,1,33.0,NaN,Wedding,Watching Movie
578,LP002872,No,0.0,Graduate,32297,110.0,360.0,1.0,0,23.0,Male,Wedding,Watching Movie
579,LP002872,Yes,1.0,NaN,41090,107.0,360.0,1.0,0,47.0,Male,Wedding,Watching Movie


In [ ]:
df3.drop_duplicates('UID', keep='first', inplace=True)  # drop duplicates('Column', keep='first')
df3.shape    # shape of the DataFrame after dropping duplicates (2 duplicates)

(612, 13)

In [ ]:
# Data standardization
df3['Marital_status'].value_counts() # counts the occurrences of each unique value

,count
Marital_status,
Yes,273
No,155
YES,112
NO,70


In [ ]:
df3['Marital_status'] = df3['Marital_status'].str.upper()   # converts all the values to Uppercase
df3['Marital_status'].value_counts()   # counts the occurrences of 'YES' and 'NO' after case conversion

,count
Marital_status,
YES,385
NO,225


In [ ]:
df3['Sex'].value_counts()    # counts the occurrences of each unique value

,count
Sex,
Male,460
Female,107
M,18
F,14


In [ ]:
# We replace 'M' -> 'Male' and 'F' -> 'Female'
df3['Sex'] = df3['Sex'].replace({'M' : 'Male', 'F' : 'Female'})    # replace({old_value : new_value}) dict(old_value=new_value)
df3['Sex'].value_counts()    # counts the occurrences of 'Male' and 'Female' after replacement

,count
Sex,
Male,478
Female,121


In [ ]:
# Now that we have familiarized ourselves with the data, maybe we can tackle inaccurate records?
print(df3.describe())   # Checking for Incorrect Records
print("~~~~~~~~~~~~~~~~~~")
print(df3.shape[0])   # No of rows
print("~~~~~~~~~~~~~~~~~~")
df3.isnull().sum()

       Dependents        Income  Loan_amount  Term_months  Credit_score  \
count  597.000000  6.120000e+02   590.000000   598.000000    562.000000   
mean     0.763819  1.904095e+05   146.540678   341.939799      0.841637   
std      1.016391  4.041188e+06    85.703902    65.221050      0.365406   
min      0.000000  5.001000e+03     9.000000    12.000000      0.000000   
25%      0.000000  1.608275e+04   100.000000   360.000000      1.000000   
50%      0.000000  2.623350e+04   128.000000   360.000000      1.000000   
75%      2.000000  3.839825e+04   168.000000   360.000000      1.000000   
max      3.000000  1.000000e+08   700.000000   480.000000      1.000000   

       approval_status         Age  
count       612.000000  606.000000  
mean          0.490196   38.074257  
std           0.500313   10.473079  
min           0.000000  -12.000000  
25%           0.000000   29.000000  
50%           0.000000   37.000000  
75%           1.000000   47.000000  
max           1.000000   55.

,0
UID,0
Marital_status,2
Dependents,15
Is_graduate,1
Income,0
Loan_amount,22
Term_months,14
Credit_score,50
approval_status,0
Age,6


There are two suspicious things going on with the Age column that speaks to both accuracy and missing data. Who is -12 years old? How is that possible? Plus there are 6 Ages that we don't have at all.

The strategy for Age might be to replace the missing data and the inaccurate date (let's say any age under 10) with the mean value of the other values in that column. This strategy is worked below:

In [ ]:
# Why is there a minimum age tht is -12? Who is -12 years old? We will need to remove this:
print(df3['Age'].min())   # Incorrect Minimum age
# we will replace all ages that are < 10 with a missing value and then we will address missing values in the next cell
df3.loc[df3["Age"]<10,"Age"]=df3["Age"].mean()
#check to see the min of the Age column
df3['Age'].min()
# That's better! Let's replace the 6 missing values with the mean value of the column, too.
#df3['Age'].fillna(df3['Age'].mean(),inplace=True)
df3["Age"] = df3["Age"].fillna(df3["Age"].mean())
# You can also use the SimpleImputer function from sklearn
# as we used in the df1, df2 examples.

-12.0


In [ ]:
print(df3["Age"])
# have we replaced all the missing elements in the Age column with the mean?
df3["Age"].isnull().sum()

0      40.0
2      22.0
3      27.0
4      36.0
5      55.0
       ... 
610    55.0
611    47.0
612    27.0
613    47.0
614    42.0
Name: Age, Length: 612, dtype: float64


np.int64(0)

In [ ]:
# handle missing values:
print(df3.shape)
print("--------------")
df3.isnull().sum()

(612, 13)
--------------


,0
UID,0
Marital_status,2
Dependents,15
Is_graduate,1
Income,0
Loan_amount,22
Term_months,14
Credit_score,50
approval_status,0
Age,0


In [ ]:
# replace missing (null or NaN) values with mean of that column
df3['Loan_amount'].fillna(df3['Term_months'].mean())
#df3.isnull().sum()
df3['Term_months'].fillna(df3['Term_months'].mean())
print("*****************")
df3.isnull().sum()

*****************


,0
UID,0
Marital_status,2
Dependents,15
Is_graduate,1
Income,0
Loan_amount,22
Term_months,14
Credit_score,50
approval_status,0
Age,0


In [ ]:
# we might decide that some of the columns contain irrelevant information
# so we can just drop all the rows with missing values
df3.dropna(inplace=True)   # We drop the records with missing values
print(df3.isnull().sum())   # count missing values
df3.shape # we can see how many rows we have now dropped.

UID                0
Marital_status     0
Dependents         0
Is_graduate        0
Income             0
Loan_amount        0
Term_months        0
Credit_score       0
approval_status    0
Age                0
Sex                0
Purpose            0
Hobby              0
dtype: int64


(502, 13)